In [1]:
key = 'qQYVw6u9MbLH35Tt1qHSsNpAGM6QFz8x'

In [2]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

In [3]:
# Specifying the date range
end = datetime.date.today()
start = end - relativedelta(years=50)

In [4]:
months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]

In [5]:
print(months_in_range)

[['1971', '8'], ['1971', '9'], ['1971', '10'], ['1971', '11'], ['1971', '12'], ['1972', '1'], ['1972', '2'], ['1972', '3'], ['1972', '4'], ['1972', '5'], ['1972', '6'], ['1972', '7'], ['1972', '8'], ['1972', '9'], ['1972', '10'], ['1972', '11'], ['1972', '12'], ['1973', '1'], ['1973', '2'], ['1973', '3'], ['1973', '4'], ['1973', '5'], ['1973', '6'], ['1973', '7'], ['1973', '8'], ['1973', '9'], ['1973', '10'], ['1973', '11'], ['1973', '12'], ['1974', '1'], ['1974', '2'], ['1974', '3'], ['1974', '4'], ['1974', '5'], ['1974', '6'], ['1974', '7'], ['1974', '8'], ['1974', '9'], ['1974', '10'], ['1974', '11'], ['1974', '12'], ['1975', '1'], ['1975', '2'], ['1975', '3'], ['1975', '4'], ['1975', '5'], ['1975', '6'], ['1975', '7'], ['1975', '8'], ['1975', '9'], ['1975', '10'], ['1975', '11'], ['1975', '12'], ['1976', '1'], ['1976', '2'], ['1976', '3'], ['1976', '4'], ['1976', '5'], ['1976', '6'], ['1976', '7'], ['1976', '8'], ['1976', '9'], ['1976', '10'], ['1976', '11'], ['1976', '12'], ['1977

In [6]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + 'qQYVw6u9MbLH35Tt1qHSsNpAGM6QFz8x'
    response = requests.get(url).json()
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        response = send_request(date)
        df = parse_response(response)
        total += len(df)
        df.to_csv('headlines/' + date[0] + '-' + date[1] + '.csv', index=False)
        print('Saving headlines/' + date[0] + '-' + date[1] + '.csv...')
    print('Number of articles collected: ' + str(total))


In [7]:
get_data(months_in_range)

Date range: ['1971', '8'] to ['2021', '7']
Saving headlines/1971-8.csv...
Saving headlines/1971-9.csv...
Saving headlines/1971-10.csv...
Saving headlines/1971-11.csv...
Saving headlines/1971-12.csv...
Saving headlines/1972-1.csv...
Saving headlines/1972-2.csv...
Saving headlines/1972-3.csv...
Saving headlines/1972-4.csv...
Saving headlines/1972-5.csv...
Saving headlines/1972-6.csv...
Saving headlines/1972-7.csv...
Saving headlines/1972-8.csv...
Saving headlines/1972-9.csv...
Saving headlines/1972-10.csv...
Saving headlines/1972-11.csv...
Saving headlines/1972-12.csv...
Saving headlines/1973-1.csv...
Saving headlines/1973-2.csv...
Saving headlines/1973-3.csv...
Saving headlines/1973-4.csv...
Saving headlines/1973-5.csv...
Saving headlines/1973-6.csv...
Saving headlines/1973-7.csv...
Saving headlines/1973-8.csv...
Saving headlines/1973-9.csv...
Saving headlines/1973-10.csv...
Saving headlines/1973-11.csv...
Saving headlines/1973-12.csv...
Saving headlines/1974-1.csv...
Saving headlines/1

Saving headlines/1993-5.csv...
Saving headlines/1993-6.csv...
Saving headlines/1993-7.csv...
Saving headlines/1993-8.csv...
Saving headlines/1993-9.csv...
Saving headlines/1993-10.csv...
Saving headlines/1993-11.csv...
Saving headlines/1993-12.csv...
Saving headlines/1994-1.csv...
Saving headlines/1994-2.csv...
Saving headlines/1994-3.csv...
Saving headlines/1994-4.csv...
Saving headlines/1994-5.csv...
Saving headlines/1994-6.csv...
Saving headlines/1994-7.csv...
Saving headlines/1994-8.csv...
Saving headlines/1994-9.csv...
Saving headlines/1994-10.csv...
Saving headlines/1994-11.csv...
Saving headlines/1994-12.csv...
Saving headlines/1995-1.csv...
Saving headlines/1995-2.csv...
Saving headlines/1995-3.csv...
Saving headlines/1995-4.csv...
Saving headlines/1995-5.csv...
Saving headlines/1995-6.csv...
Saving headlines/1995-7.csv...
Saving headlines/1995-8.csv...
Saving headlines/1995-9.csv...
Saving headlines/1995-10.csv...
Saving headlines/1995-11.csv...
Saving headlines/1995-12.csv...

Saving headlines/2015-4.csv...
Saving headlines/2015-5.csv...
Saving headlines/2015-6.csv...
Saving headlines/2015-7.csv...
Saving headlines/2015-8.csv...
Saving headlines/2015-9.csv...
Saving headlines/2015-10.csv...
Saving headlines/2015-11.csv...
Saving headlines/2015-12.csv...
Saving headlines/2016-1.csv...
Saving headlines/2016-2.csv...
Saving headlines/2016-3.csv...
Saving headlines/2016-4.csv...
Saving headlines/2016-5.csv...
Saving headlines/2016-6.csv...
Saving headlines/2016-7.csv...
Saving headlines/2016-8.csv...
Saving headlines/2016-9.csv...
Saving headlines/2016-10.csv...
Saving headlines/2016-11.csv...
Saving headlines/2016-12.csv...
Saving headlines/2017-1.csv...
Saving headlines/2017-2.csv...
Saving headlines/2017-3.csv...
Saving headlines/2017-4.csv...
Saving headlines/2017-5.csv...
Saving headlines/2017-6.csv...
Saving headlines/2017-7.csv...
Saving headlines/2017-8.csv...
Saving headlines/2017-9.csv...
Saving headlines/2017-10.csv...
Saving headlines/2017-11.csv...
